In [79]:
#import libraries needed
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

# Optimization via Stochastic Gradient Descent

While working with Machine Learning (ML) you are usually given a dataset $D = {X, Y}$ with $X = [x^1 x^2 ... x^N] \in \mathbb{R^{dxN}}$ and $Y = [y^1 y^2 ... y^N] \in \mathbb{R^N}$ and a parametric function $f_w(x)$ where the vector $w$ is sually referred to as the weights of the model. The training procedure can be written as

$$ w^*=\underset{w}{arg\,min}\;l(w; \mathbb{D}) = \underset{w}{arg\,min\;\sum^N_{i=1} l_i(w; x^{(i)},y^{(i)})}$$

what is interesting from the optimization point of view, is that the objective function l(w; D) is written as a sum of independent terms that are related to datapoints (we will see in the next lab why this formulation is so common).

Suppose we want to apply GD. Given an initial vector $w_0 ∈ \mathbb{R}^n$, the iteration become

$$w_{k+1}=w_k-\alpha_k \nabla_wl(w_k;\mathbb{D})=w_k-\alpha_k \sum_{i=1}^N \nabla_wl(w_k;x^{(i)},y^{(i)})$$

Thus, to compute the iteration we need the gradient with respect to the weights of the objective functions, that can be computed by summing up the gradients of the independent functions $l_i(w;x^{(i)},y^{(i)})$.

Unfortunately, even if it is easy to compute the gradient for each of the $l_i(w;x^{(i)},y^{(i)})$, when the number of samples N is large (which is common in Machine Learning), the computation of the full gradient $∇_wl(w_k; D)$ is prohibitive. For this reason, in such optimization problems, instead of using a standard GD algorithm, it is better using the Stochastic Gradient Descent (SGD) method. That is a variant of the classical GD where, instead of computing $∇_wl(w; D) = \sum^N_{i=1} ∇_wl_i(w; x^{(i)}, y^{(i)})$, the summation is reduced to a limited numberof terms, called a batch. The idea is the following:

- Given a number $N_{batch}$ (usually called batch size), randomly extract a subdataset $M$ with $|M| = N_{batch}$ from $\mathbb{D}$.

- Approximate the true gradient 
    $$∇_wl(w; D) = \sum^N_{i=1} ∇_wl_i(w; x^{(i)}, y^{(i)})$$ 
    with 
    $$∇_wl(w; M) = \sum_{i∈\mathbb{M}} ∇_wl_i(w; x^{(i)}, y^{(i)})$$

- Compute one single iteration of the GD algorithm
$$w_{k+1} = w_k − α_k∇_wl(w;M)$$

- Repeat until you have extracted the full dataset. Notice that the random sampling at each iteration is done without replacement.

Each iteration of the algorithm above is usually called batch iteration. When the whole dataset has been
processed, we say that we completed an epoch of the SGD method. This algorithm should be repeated for e
fixed number E of epochs to reach convergence.

Unfortunately, one of the biggest drawbacks of SGD with respect to GD, is that now we cannot check the
convergence anymore (since we can’t obviously compute the gradient of $l(w; D)$ to check its distance from
zero) and we can’t use the backtracking algorithm, for the same reason. As a consequence, the algorithm
will stop ONLY after reaching the fixed number of epochs, and we must set a good value for the step size
αk by hand. Those problems are solved by recent algorithms like SGD with Momentum, Adam, AdaGrad, ...

## Implement SGD function

Write a Python script that implement the SGD algorithm, following the structure you already wrote
for GD. That script should work as follows:

    Input:
    - l: the function l(w; D) we want to optimize.
        It is supposed to be a Python function, not an array.
    - grad_f: the gradient of l(w; D). 
        It is supposed to be a Python function, not an array.
    - w0: an n-dimensional array which represents the initial iterate. 
        By default, it should be randomly sampled.
    - data: a tuple (x, y) that contains the two arrays x and 
            y, where x is the input data, y is the output data.
    - batch_size: an integer. 
        The dimension of each batch. Should be a divisor of the number of data.
    - n_epochs: an integer. The number of epochs you want to 
                reapeat the iterations.
    
    Output:
    - w: an array that contains the value of w_k FOR EACH   
        iterate w_k (not only the latter).
    - f_val: an array that contains the value of l(w_k; D)
         FOR EACH iterate w_k ONLY after each epoch.
    - grads: an array that contains the value of grad_l(w_k;D) 
        FOR EACH iterate w_k ONLY after each epoch.
    - err: an array the contains the value of ||grad_l(w_k; D)||_2 
        FOR EACH iterate w_k ONLY after each epoch.

In [80]:
def shuffle_data(X, Y, size):
    _, N = X.shape
    indexes = np.arange(N)
    np.random.shuffle(indexes)

    X_shuffle = X[:, indexes]
    Y_shuffle = Y[indexes]

    return X_shuffle, Y_shuffle

In [81]:
def SGD(l, grad_l, w0, D, batch_size, n_epochs, alpha):
    lr = alpha  
    X, Y = D  #Split D into X and Y
    d, N = X.shape

    n_batch_per_epoch = N//batch_size
    tot_batch = n_batch_per_epoch * n_epochs
    
    w = np.array(w0)
    f_val = np.zeros((n_epochs, ))
    grads = np.zeros((n_epochs, d))
    err = np.zeros((n_epochs,))
    w_vector = np.zeros((tot_batch, len(w0)))
    
    #For each epoch    
    for epoch in range(n_epochs):
        X_shuffle, Y_shuffle = shuffle_data(X, Y, N//batch_size)
        
        for b in range (n_batch_per_epoch):  
            #Sample M from D
            n = b*batch_size
            m = (b+1)*batch_size

            Mx = X_shuffle[:, n:m]
            My = Y_shuffle[n:m]

            #Mx <- X    Mx batch from x  --> shape d x batch_size
            #My <- Y    My batch from y  --> shape batch_size

            #Update w
            w = w - lr * grad_l(w, Mx, My)
            w_vector[epoch*n_batch_per_epoch + b, :] = w
            

        f_val[epoch] = l(w, X_shuffle, Y_shuffle)
        grads[epoch, :] = grad_l(w, X_shuffle, Y_shuffle)
        err[epoch] = np.linalg.norm(grad_l(w, X_shuffle, Y_shuffle))

        ## ATTENTION: you have to shuffle again (differently)

    return w_vector, f_val, grads, err


#REMEMBER: in SG, w0 should be chosen randomly (sample from Gaussian)


## Prepare Dataset and Loss

• To test the script above, consider the MNIST dataset we used in the previous laboratories, and do the following:

1. From the dataset, select only two digits. It would be great to let the user input the two digits to select.
2. Do the same operation of the previous homework to obtain the training and test set from (X, Y), selecting The $N_{train}$ you prefer.
3. Implement a logistic regression classificator as described in the corresponding post on my website.

In [82]:
#Load data into memory
data = pd.read_csv('./data.csv')

#Convert data into a matrix
data = np.array(data)   

X0 = data[:, 1:].T
Y0 = data[:, 0]

def choose_labels(labels):
    idx = [index for index, elem in enumerate(Y0) if elem in labels]

    X = X0[:, idx]     
    Y = Y0[idx]

    return X, Y

def split_data(X, Y, Ntrain):

    d, N = X.shape

    idx = np.arange(N)
    np.random.shuffle(idx)

    train_idx = idx[:Ntrain]
    test_idx = idx[Ntrain:]

    Xtrain = X[:, train_idx]
    Ytrain = Y[train_idx]
    
    Xtest = X[:, test_idx]
    Ytest = Y[test_idx]

    return (Xtrain, Ytrain), (Xtest, Ytest)



In [83]:
# Value of the loss
def loss_function(w, x_hat, y):
    z = fw(x_hat, w)
    
    _, N = x_hat.shape
    y = np.reshape(y, (N,1))
    
    return (np.sum(MSE(z, y))/N)

def fw(xhat, w):
    return sigmoid(xhat.T @ w)

def sigmoid(z):
    return 1./(1 + np.exp(-z))

def MSE(y, y1):
    return ((np.linalg.norm(y - y1))**2)    #AXIS=1????kil

# Value of the gradient
def grad_loss_function(w, x_hat, y):
    _, N = x_hat.shape

    z = fw(w, x_hat) #row vector

    #transform to column vectors
    y = np.reshape(y, (N,1))
    z = np.reshape(z, (N,1))

    grad_mse = z * (1-z) * x_hat.T * (z-y)
    return (np.sum(grad_mse))/N


#Prediction over new data
def predict(w, X_hat, treshold=0.5):
    z = fw(w, X_hat)
    
    _, N = X_hat.shape
    predictions = np.zeros((N,))
    predictions = z>treshold
    return predictions

def accuracy(pred, label):
    N, = pred.shape
    correct = [pred == label]
    correct_sum = int(np.sum(correct))

    return correct_sum/N* 100
    

In [84]:
label1 = int(input("Choose a digit: "))
label2 = int(input("Choose another digit: "))
classes = [label1, label2]

X, Y = choose_labels(classes)


# Binary conversion 
# the labels must be binary
Y[Y == classes[0]] = 0
Y[Y == classes[1]] = 1

Ntrain = int(0.9 * Y.shape[0])
(Xtrain, Ytrain), (Xtest, Ytest) = split_data(X, Y, Ntrain)

#Fitting the model

l, N = Xtrain.shape

# Computation of the dataset with ones in the first element
X_hat = np.concatenate((np.ones((1, N)), Xtrain))

#Initial weights vector
mean=0
sigma=1e-3
w0 = np.random.normal(mean, sigma, size=Xtrain.shape[0]+1)

D = (X_hat, Ytrain)

#Batch size
batch_size = 8

#Number of epochs
n_epochs = 1

#Learning rate
alpha = 1e-4

#Training
w_vector, fval, gradl, err = SGD(loss_function, grad_loss_function, w0, D, batch_size, n_epochs, alpha)

#Testing the accuracy on test data
X_hat_test = np.concatenate((np.ones((1, Xtest.shape[1])), Xtest))

#select the final weights from the list
final_weights = w_vector[-1]

predictions = predict(final_weights, X_hat_test)

acc = accuracy(predictions, Ytest)
print(f"The accuracy is: {acc}%")

The accuracy is: 52.313167259786475%


- Test the logistic regression classificator for different digits and different training set dimensions.

- The training procedure will end up with a set of optimal parameters $w^*$. Compare $w^*$ when computed with GD and SGD, for different digits and different training set dimensions.

- Comment the obtained results (in terms of the accuracy of the learned classificator).

- _Hard (optional)_: Try to implement the 3-digits logistic regression classificator and compare its accuracy with the accuracy of LDA and PCA classificators.